In [280]:
import pandas as pd
import os
import numpy as np
import sys
sys.path.append('..')

In [282]:
os.chdir(r'C:\Users\Administrator\Desktop\raphi_other\repositories\template_project\src')

In [359]:
data = pd.read_csv(r'../data/interim/cars_data.csv')

In [286]:
#test = pd.read_csv(r'../data/interim/test.csv')

In [361]:
x = data.drop(columns=['product_tier'])
y = data.product_tier

In [363]:
# x['ctr'] = x['ctr'].astype('float64')

In [365]:
numerical_df = data.select_dtypes(exclude=['object'])
categorical_df = data.select_dtypes(include=['object'])

In [366]:
y

0        Basic
1        Basic
2        Basic
3        Basic
4        Basic
         ...  
48803    Basic
48804    Basic
48805    Basic
48806    Basic
48807    Basic
Name: product_tier, Length: 48808, dtype: object

In [369]:
#x.to_csv('../data/interim/house_xinit.csv', index=False)
#y.to_csv('../data/interim/house_yinit.csv', index=False)

In [370]:
#y.value_counts()

# Ordinal Encoding of y

In [372]:
order = ['Basic', 'Premium', 'Plus']#, 'Overweight_Level_II', 'Obesity_Type_I', 'Obesity_Type_II', 'Obesity_Type_III']

# Convert 'y' to a categorical type with the specified order
y = pd.Categorical(y, categories=order, ordered=True).codes

In [375]:
y = pd.Series(y, name='y')

In [377]:
y

0        0
1        0
2        0
3        0
4        0
        ..
48803    0
48804    0
48805    0
48806    0
48807    0
Name: y, Length: 48808, dtype: int8

In [379]:
numerical_df = x.select_dtypes(exclude=['object'])
categorical_df = x.select_dtypes(include=['object'])

In [381]:
numerical_df

,price,first_zip_digit,search_views,detail_views,stock_days,live_days
0,16750,5,3091.0,123.0,30,31
1,1750,6,1856.0,26.0,101,101
2,24890,6,125.0,6.0,2,2
3,2850,7,172.0,4.0,1,31
4,23740,2,1499.0,99.0,87,87
...,...,...,...,...,...,...
48803,3999,8,7149.0,189.0,50,139
48804,3297,3,604.0,34.0,4,4
48805,2950,6,1175.0,16.0,25,25
48806,7850,8,448.0,21.0,16,224


In [383]:
categorical_df

,make_name
0,Mitsubishi
1,Ford
2,Volkswagen
3,Fiat
4,Renault
...,...
48803,BMW
48804,Ford
48805,Hyundai
48806,Volkswagen


In [386]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in x.columns if x[cname].nunique() < 800 and x[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in x.columns if x[cname].dtype in ['int64', 'float64']]

In [388]:
categorical_cols

['make_name']

In [390]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler, MinMaxScaler

#train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [316]:
# X_train.to_csv('../data/interim/x_train_init.csv', index=False)
# X_test.to_csv('../data/interim/x_test_init.csv', index=False)
# y_train.to_csv('../data/interim/y_train_init.csv', index=False)
# y_test.to_csv('../data/interim/y_test_init.csv', index=False)

In [318]:
X_train.head()

,make_name,price,first_zip_digit,search_views,detail_views,stock_days,live_days
31741,Audi,23900,3,854.0,20.0,32,32
36475,Mitsubishi,14900,3,2818.0,88.0,7,244
24577,Citroen,1400,5,1891.0,64.0,28,28
16079,Renault,12950,3,126.0,1.0,28,28
23422,Ford,8950,1,598.0,31.0,32,32


In [320]:
print(y_train)

31741    0
36475    1
24577    0
16079    0
23422    0
        ..
11284    0
44732    0
38158    0
860      0
15795    0
Name: y, Length: 34165, dtype: int8


In [321]:
#y_train = np.log(y_train)

In [324]:
#y_test = np.log(y_test)

In [325]:
print(y_train)

31741    0
36475    1
24577    0
16079    0
23422    0
        ..
11284    0
44732    0
38158    0
860      0
15795    0
Name: y, Length: 34165, dtype: int8


In [222]:
from category_encoders import BaseNEncoder

In [392]:
from sklearn.preprocessing import FunctionTransformer


# Define the log transformation function
#log_transformer = FunctionTransformer(np.log, 
#                                      inverse_func=np.exp, 
#                                      validate=True)

shifted_log_transformer = Pipeline(steps=[
    ('shift', FunctionTransformer(lambda x: x + 1)),  # Add 1 to avoid log(0)
    ('log', FunctionTransformer(np.log))
])

# Columns to apply log transformation
log_columns = ['price', 'search_views', 'detail_views', 'live_days']

In [394]:
#create numerical transformer



numerical_transformer = Pipeline([('imputer', SimpleImputer(strategy='mean')), 
                                  ('scaler', StandardScaler()) ])

#create categorical transformer
categorical_transformer = Pipeline(steps=[ ('imputer', SimpleImputer(strategy='most_frequent')),
                                            ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                            ])

target_encoding_pipeline = Pipeline(steps=[
    ('base_encoder', BaseNEncoder(cols=['make_name'], base=3))
])

In [396]:
# Combine the transformations using ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('log', shifted_log_transformer, log_columns),
    ('base_name', target_encoding_pipeline, ['make_name']),  # TargetEncoder for 'town'
    ('num', numerical_transformer, numerical_cols),       # Transformer for numerical columns
    ('cat_onehot', categorical_transformer, categorical_cols) # OneHotEncoder for other categorical features
])

In [398]:
# #column transformer
# preprocessor = ColumnTransformer(
#                                 transformers=[
#                                     ('num', numerical_transformer, numerical_cols),
#                                     ('cat', categorical_transformer, categorical_cols)
#     ])

# Upsampling

In [ ]:
# from sklearn.utils import resample

# sampling_strategy = {
#     0: y_train.value_counts()[0],  # Keep class 0 as is
#     1: y_train.value_counts()[0],  # Oversample class 1 to the same as class 0
#     2: y_train.value_counts()[0]   # Oversample class 2 to the same as class 0
# }

# # Example of undersampling
# oversampler = RandomOverSampler(random_state=42, sampling_strategy=sampling_strategy)
# X_train, y_train = oversampler.fit_resample(X_train, y_train)

In [409]:
from sklearn.utils import resample

# Combine features and target into a single DataFrame
df_train = pd.concat([X_train, y_train], axis=1)

# Rename the target column for clarity (if not already named)
# Replace 'target' with your actual target column name if different
df_train.columns = list(X_train.columns) + ['product_tier']

# Identify the majority class (assumed to be class 0)
majority_class = df_train[df_train['product_tier'] == 0]

# Identify minority classes
minority_class1 = df_train[df_train['product_tier'] == 1]
minority_class2 = df_train[df_train['product_tier'] == 2]

n_majority = len(majority_class)
print(f"Number of samples in majority class (Class 0): {n_majority}")

# Oversample Class 1
minority_class1_upsampled = resample(
    minority_class1,
    replace=True,           # Sample with replacement
    n_samples=n_majority,  # Match number of majority class
    random_state=42         # For reproducibility
)

# Oversample Class 2
minority_class2_upsampled = resample(
    minority_class2,
    replace=True,
    n_samples=n_majority,
    random_state=42
)

# Combine majority class with oversampled minority classes
df_upsampled = pd.concat([majority_class, minority_class1_upsampled, minority_class2_upsampled])

# Shuffle the dataset to ensure random distribution
df_upsampled = df_upsampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Separate features and target
X_train_upsampled = df_upsampled.drop('product_tier', axis=1)
y_train_upsampled = df_upsampled['product_tier']

print("Class distribution before oversampling:")
print(y_train.value_counts())

print("\nClass distribution after oversampling:")
print(y_train_upsampled.value_counts())

X_train = X_train_upsampled #= df_upsampled.drop('product_tier', axis=1)
y_train = y_train_upsampled #= df_upsampled['product_tier']

Number of samples in majority class (Class 0): 33483
Class distribution before oversampling:
y
0    33483
1      518
2      164
Name: count, dtype: int64

Class distribution after oversampling:
product_tier
2    33483
1    33483
0    33483
Name: count, dtype: int64


In [340]:
X_train.make_name.value_counts()

make_name
Volkswagen    11866
Renault        9047
Volvo          7239
Peugeot        6556
Ford           6232
              ...  
Daimler           1
Casalini          1
Buick             1
FISKER            1
Kawasaki          1
Name: count, Length: 72, dtype: int64

In [342]:
# from imblearn.under_sampling import RandomUnderSampler

# # Example of undersampling
# undersampler = RandomUnderSampler(random_state=42, sampling_strategy='majority')
# X_train, y_train = undersampler.fit_resample(X_train, y_train)

In [344]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Create a RandomForestClassifier model
rf = RandomForestClassifier(n_estimators=150, random_state=42, max_depth=4)

# Model pipeline
rf_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('random_forest', rf)
                         ])

# Preprocessing of training data, fit model 
rf_pipe.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
rf_preds = rf_pipe.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, rf_preds)
print('Accuracy for Random Forest Model:', accuracy)

# Detailed classification report
print('Classification Report:\n', classification_report(y_test, rf_preds))


Accuracy for Random Forest Model: 0.7290172778802158
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.73      0.84     14385
           1       0.10      0.54      0.17       181
           2       0.01      0.35      0.02        77

    accuracy                           0.73     14643
   macro avg       0.37      0.54      0.34     14643
weighted avg       0.98      0.73      0.83     14643



In [236]:
rf_preds = rf_pipe.predict(X_train)

# Evaluate the model
accuracy = accuracy_score(y_train, rf_preds)
print('Accuracy for Random Forest Model:', accuracy)

# Detailed classification report
print('Classification Report:\n', classification_report(y_train, rf_preds))


Accuracy for Random Forest Model: 0.6367345532237499
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.73      0.66     48365
           1       0.68      0.61      0.65     48365
           2       0.63      0.57      0.60     48365

    accuracy                           0.64    145095
   macro avg       0.64      0.64      0.64    145095
weighted avg       0.64      0.64      0.64    145095



In [238]:
import plotly.graph_objects as go
# Get feature names from the preprocessor
feature_names = preprocessor.get_feature_names_out()

importances = rf.feature_importances_

# Get the feature importances from the Random Forest model
# Create a dictionary mapping feature names to their importances
feature_importance_dict = dict(zip(feature_names, importances))

# Optional: Sort the dictionary by importance in descending order
feature_importance_sorted = dict(sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True))

fig = go.Figure(data=[
        go.Bar(
            x=list(feature_importance_sorted.keys()),
            y=list(feature_importance_sorted.values()),
            marker=dict(color='skyblue'),
            text=[f"{v:.3f}" for v in feature_importance_sorted.values()],
            textposition='auto'
        )
    ])

# Update layout for better aesthetics
fig.update_layout(
    title='Feature Importances from Random Forest',
    xaxis_title='Features',
    yaxis_title="Importance Score",
    template='plotly_white',
    height=600
)

# Display the figure
fig.show()

AttributeError: Estimator shift does not provide get_feature_names_out. Did you mean to call pipeline[:-1].get_feature_names_out()?

In [ ]:
#show decision tree
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (80,40)
tree.plot_tree(rf,filled = True);
#plt.savefig('./images/tree_example.png')

In [463]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
# XGBoost
xgb = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.3,
    max_depth=4,
    random_state=42,
    subsample=0.7,
    gamma=3,  # Increased from default
    reg_alpha=8,    # Added L1 regularization
    #reg_lambda=3,   # Added L2 regularization (can also be increased)
    eval_metric='mlogloss'
)

# Model pipeline
rf_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xgboost', xgb)
])

# Preprocessing of training data, fit model 
rf_pipe.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
rf_preds = rf_pipe.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, rf_preds)
print('Accuracy for XGBoost:', accuracy)

# Detailed classification report
print('Classification Report:\n', classification_report(y_test, rf_preds))

Accuracy for XGBoost: 0.9296592228368503
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.94      0.96     14385
           1       0.17      0.69      0.27       181
           2       0.04      0.18      0.07        77

    accuracy                           0.93     14643
   macro avg       0.40      0.60      0.43     14643
weighted avg       0.98      0.93      0.95     14643



In [449]:
rf_preds = rf_pipe.predict(X_train)

# Evaluate the model
accuracy = accuracy_score(y_train, rf_preds)
print('Accuracy for Random Forest Model:', accuracy)

# Detailed classification report
print('Classification Report:\n', classification_report(y_train, rf_preds))


Accuracy for Random Forest Model: 0.9836633515515336
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.95      0.97     33483
           1       0.97      1.00      0.98     33483
           2       0.98      1.00      0.99     33483

    accuracy                           0.98    100449
   macro avg       0.98      0.98      0.98    100449
weighted avg       0.98      0.98      0.98    100449

